In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install spacy-wordnet

In [ ]:
!python -m nltk.downloader wordnet
!python -m nltk.downloader omw

In [ ]:
import spacy

from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

# Load an spacy model (supported models are "es" and "en") 
nlp = spacy.load('en')
nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
token = nlp('prices')[0]

# wordnet object link spacy token with nltk wordnet interface by giving acces to
# synsets and lemmas 
token._.wordnet.synsets()
token._.wordnet.lemmas()

# And automatically tags with wordnet domains
token._.wordnet.wordnet_domains()

In [ ]:
economy_domains = ["social"]
enriched_sentence = []
sentence = nlp('I will kill you all')

# For each token in the sentence
for token in sentence:
    # We get those synsets within the desired domains
    synsets = token._.wordnet.wordnet_synsets_for_domain(economy_domains)
    if not synsets:
        enriched_sentence.append(token.text)
    else:
        lemmas_for_synset = [lemma for s in synsets for lemma in s.lemma_names()]
        # If we found a synset in the economy domains
        # we get the variants and add them to the enriched sentence
        enriched_sentence.append('({})'.format('|'.join(set(lemmas_for_synset))))

# Let's see our enriched sentence
print(' '.join(enriched_sentence))
# >> I (need|want|require) to (draw|withdraw|draw_off|take_out) 5,000 euros
    

In [ ]:
! pip install profanity_filter

In [ ]:
from profanity_filter import ProfanityFilter

pf = ProfanityFilter()

pf.censor("I will fuck you all")
# "That's ********!"

#pf.censor_word('fuck')
# Word(uncensored='fuck', censored='****', original_profane_word='fuck')

In [ ]:
from nltk.corpus import wordnet
wordnet.synsets('destroy') 

In [ ]:
synonyms = []
for syn in wordnet.synsets("destroy"): 
    for l in syn.lemmas(): 
        synonyms.append(l.name()) 
print(set(synonyms))

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('/kaggle/input/bin_sentiment_try2.h5')

In [ ]:
model.summary()

In [ ]:
#!pip install simplejson
import simplejson as json

In [ ]:
from keras.preprocessing.text import tokenizer_from_json
with open('/kaggle/input/tokenizer (4).json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
bad_dict = {}
for word in synonyms:
    seq = tokenizer.texts_to_sequences(list(word))
    pad = pad_sequences(seq, maxlen=800, padding='post')
    x = model.predict(pad)
    print("word {} toxicity {}".format(word,x[0][0]))
    bad_dict[word] = x[0][0]

In [ ]:
print(bad_dict)

In [ ]:
#x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
{k: v for k, v in sorted(bad_dict.items(), key=lambda item: item[1],reverse=True)}
#{0: 0, 2: 1, 1: 2, 4: 3, 3: 4}